# COMP5046 Assignment 1
*Make sure you change the file name with your unikey.*

# Readme
*If there is something to be noted for the user, please mention here.* 

*If you are planning to implement a program with Object Oriented Programming style*

***Visualising the comparison of different results is a good way to justify your decision.***

# 1 - Data Preprocessing (Personality chat datasets)

## 1.1. Download Dataset (Personality chat datasets)

In [0]:
# Please comment your code

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# links of 3 csv files
link_professional = 'https://drive.google.com/open?id=1oPxUI1xxdye5-tIdokYjLvYIRfFOhGD-' 
link_friend = 'https://drive.google.com/open?id=1TZTWQnfQaySF72PRxp3PEuZ8kdF4v_Ti' 
link_comic = 'https://drive.google.com/open?id=1BukHDfwEbxxK8Df7jWRhXIDksjyf8HWq' 

# names of files
names=['qna_chitchat_the_professional.csv', 'qna_chitchat_the_friend.csv', 'qna_chitchat_the_comic.csv']

# dictionary to store file names and corresponding content in dataframe format
dic={}

# store links to obtain relevant ids
files = [link_professional, link_friend, link_comic]
for i in range(0, len(files)):
    fluff, id = files[i].split('=') 
    downloaded = drive.CreateFile({'id': id})
    downloaded.GetContentFile(names[i])
    dic[i]=pd.read_csv(names[i], sep=',', header=0)

## 1.2. Preprocess data (Personality chat datasets)

*You are required to describe which data preprocessing techniques were conducted with justification of your decision. *

In [49]:
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords as sw
from nltk.stem.porter import *
stemmer = PorterStemmer()

nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
# Please comment your code

contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", 
                    "couldn't": "could not", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", 
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", 
                    "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", 
                    "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", 
                    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", 
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                    "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "let's": "let us", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", 
                    "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                    "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", 
                    "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", 
                    "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", 
                    "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", 
                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", 
                    "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", 
                    "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", 
                    "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", 
                    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

def preprocess(tokens, punctuations=False, decapitalisation=False, remove_numbers=False,
               stemming=False, lemmatising=False):
    #remove punctuarions and emoticons 
    if punctuations:
        tokens = [x for x in tokens if re.sub(r'[^\w\s]','',x)]
    #decapitalisation of all words in a sentence
    if decapitalisation:
        tokens = [x.lower() for x in tokens]
    #Stemming
    if stemming:
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(x) for x in tokens]
  
    #Lemmatising
    if lemmatising:
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(x) for x in tokens]
    
    return tokens



# remove puncutations

def remove_punctuation_re(x):
    x = re.sub(r'[^\w\s]','',x)
    
    return x

  
# 
Questions = {}
Answers = {}

for id, text in dic.items():
    # get questions and answers seperately
    
    Question=[]
    Answer=[]
    for i in range(len(text)):
        Question.append(text["Question"].loc[i])
        Answer.append(text["Answer"].loc[i])
        
        # replace contraction with words
        for old, new in contraction_dict.items():
            
            Question[i] = Question[i].lower().replace(old, new)
            Answer[i] = Answer[i].lower().replace(old, new)

            
        
        # tokenization
        Question[i] = word_tokenize(Question[i])
        Answer[i] = word_tokenize(Answer[i])
        
        # remove "?" and "."
        Question[i] = [token for token in Question[i] if token != "?"]
        Answer[i] = [token for token in Answer[i] if token != "."]
        
        # preprocessing tokens
        preprocess(Question[i],punctuations=True, decapitalisation=True, remove_numbers=True, 
                   stemming=True, lemmatising=True)
        preprocess(Answer[i],punctuations=True, decapitalisation=True, remove_numbers=True,
                   stemming=True, lemmatising=True)
        
        
    Questions[id]=Question
    Answers[id]=Answer





# 2 - Model Implementation

## 2.1. Word Embeddings

*You are required to describe which model was implemented (i.e. Word2Vec with CBOW, FastText with SkipGram, etc.) with justification of your decision *

In [0]:
# Please comment your code




### 2.1.1. Download Dataset for Word Embeddings



*You are required to describe which data was used with justification of your decision.*

In [0]:
# Please comment your code

### 2.1.2. Data Preprocessing for Word Embeddings

*You are required to describe which preprocessing techniques were used with justification of your decision.*

In [0]:
# Please comment your code

### 2.1.3. Build Word Embeddings Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

In [0]:
# Please comment your code

### 2.1.4. Train Word Embeddings Model

In [0]:
# Please comment your code

### 2.1.5. Save Word Embeddings Model

In [0]:
# Please comment your code

### 2.1.6. Load Word Embeddings Model

In [0]:
# Please comment your code

## 2.2. Seq2Seq model

### 2.2.1. Apply/Import Word Embedding Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

In [0]:
# Please comment your code

### 2.2.2. Build Seq2Seq Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

In [0]:
# Please comment your code

### 2.2.3. Train Seq2Seq Model

In [0]:
# Please comment your code

### 2.2.4. Save Seq2Seq Model

In [0]:
# Please comment your code

### 2.2.5. Load Seq2Seq Model

In [0]:
# Please comment your code

# 3 - Evaluation (Running chatbot)

## 3.1. Start chatting

In [0]:
# Please comment your code

## 3.2. Change Personality

*Explain how to change personality (What is the command for changing personality?). *

In [0]:
# Please comment your code

## 3.3. Save chat log

In [0]:
# Please comment your code

## 3.4. End chatting

In [0]:
# Please comment your code

## 3.5. Execute program

***Please make sure your program  is running properly.***

***Functions for downloading (from Google Drive) and loading models (both word embeddings and Seq2Seq) need to be called!*** 


### 3.5.1. Execute program - training mode

*Please include lines to train the bot.*

In [0]:
# Please comment your code


### 3.5.2. Execute program - chatting mode

*Please include lines to start chatting with the bot.*

In [0]:
# Please comment your code


## Object Oriented Programming codes here

*If you have multiple classes use multiple code snippets to add them.*

In [0]:
# If you used OOP style, use this sectioon